# Convolutional Neural Network

* Estudante: Welison Lucas Almeida Regis
* Matrícula: 17/0024121

## Mudanças implementadas

* Alteração na dimensão dos dados de entrada: `input_shape=[128,128,3]`.
* Adição de novas camadas com funções de ativação 'relu'.
* Redução de overfitting através de Dropouts.
* Armazenamento do modelo em formato ".h5".

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

     |████████████████████████████████| 320.5 MB 3.4 kB/s eta 0:00:010
Requirement already up-to-date: keras in /home/welison/.local/lib/python3.8/site-packages (2.4.3)
Requirement already up-to-date: Pillow in /home/welison/anaconda3/lib/python3.8/site-packages (7.2.0)
  Using cached numpy-1.18.5-cp38-cp38-manylinux1_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (128, 128),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (128, 128),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Adding layers to CNN

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[128, 128, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

### Output Layer

In [7]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [8]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [9]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30)

Epoch 1/30
250/250 [==============================] - 106s 425ms/step - loss: 0.7142 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5045
Epoch 2/30
250/250 [==============================] - 115s 461ms/step - loss: 0.6907 - accuracy: 0.5297 - val_loss: 0.6828 - val_accuracy: 0.5790
Epoch 3/30
250/250 [==============================] - 114s 457ms/step - loss: 0.6737 - accuracy: 0.5878 - val_loss: 0.6586 - val_accuracy: 0.6050
Epoch 4/30
250/250 [==============================] - 114s 456ms/step - loss: 0.6542 - accuracy: 0.6331 - val_loss: 0.6311 - val_accuracy: 0.6570
Epoch 5/30
250/250 [==============================] - 115s 461ms/step - loss: 0.6333 - accuracy: 0.6514 - val_loss: 0.6084 - val_accuracy: 0.6815
Epoch 6/30
250/250 [==============================] - 114s 457ms/step - loss: 0.6036 - accuracy: 0.6720 - val_loss: 0.5479 - val_accuracy: 0.7265
Epoch 7/30
250/250 [==============================] - 114s 456ms/step - loss: 0.5896 - accuracy: 0.6917 - val_loss: 0.5595 -

## Part 4 - Making a single prediction

In [10]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
if result[0][0] == training_set.class_indices['dogs']:
  prediction = 'dog'
else:
  prediction = 'cat'

In [11]:
print(prediction)

dog


## Part 5 - Save model

In [12]:
cnn.save('cats_dogs_predictor.h5')

## Fim